# Operacje na tensorach
Ta sekcja obejmuje:
* Indeksowanie i wycinanie
* Zmianę kształtu tensorów (widoki tensorów)
* Arytmetykę tensorów i operacje podstawowe
* Iloczyny skalarne
* Mnożenie macierzy
* Dodatkowe, bardziej zaawansowane operacje

## Wykonaj standardowe importy


In [1]:
import torch

## Indeksowanie i wycinanie
Wyodrębnianie konkretnych wartości z tensora działa tak samo jak w tablicach NumPy<br>
<img src='../_img/arrayslicing.png' width="500" style="display: inline-block"><br><br>
Źródło obrazu: https://scipy-lectures.org/_images/numpy_indexing.png

Na początek utwórzmy taki tensor:

In [20]:
# liczby w pierwszym wierszu
row0 = torch.arange(6)  # [0,1,2,3,4,5]

# wykorzystujemy broadcasting - reshape robi nam z wiersza kolumne z 6cioma wierszami.
x = row0 + row0.reshape(6, 1) * 10
print(x)


tensor([[ 0,  1,  2,  3,  4,  5],
        [10, 11, 12, 13, 14, 15],
        [20, 21, 22, 23, 24, 25],
        [30, 31, 32, 33, 34, 35],
        [40, 41, 42, 43, 44, 45],
        [50, 51, 52, 53, 54, 55]])


Aleternatywie

In [16]:
print(row0 + row0.unsqueeze(1) * 10)

tensor([[ 0,  1,  2,  3,  4,  5],
        [10, 11, 12, 13, 14, 15],
        [20, 21, 22, 23, 24, 25],
        [30, 31, 32, 33, 34, 35],
        [40, 41, 42, 43, 44, 45],
        [50, 51, 52, 53, 54, 55]])


`unsqueeze(1)` dodaje nowy wymiar (dimension) do tensora na pozycji 1.

W tym przypadku:
- `offsets` ma kształt `[6]` (wektor 1D): `[0, 10, 20, 30, 40, 50]`
- `offsets.unsqueeze(1)` ma kształt `[6, 1]` (macierz kolumnowa):
```
[[ 0],
 [10],
 [20],
 [30],
 [40],
 [50]]
```


To jest przydatne do operacji broadcasting - pozwala na dodawanie tego tensora kolumnowego do wektora wierszowego `row0`.

In [18]:
(row0.reshape(6,1) == row0.unsqueeze(1)).all()

tensor(True)

<img src='../_img/arrayslicing.png' width="500" style="display: inline-block"><br><br>

In [26]:
print(x[0, 3:5])
print(x[4:, 4:])
print(x[:,2])
print(x[2::2,::2])

tensor([3, 4])
tensor([[44, 45],
        [54, 55]])
tensor([ 2, 12, 22, 32, 42, 52])
tensor([[20, 22, 24],
        [40, 42, 44]])


## Zmienianie kształtu tensorów przy użyciu <tt>.view()</tt>
<a href='https://pytorch.org/docs/master/tensors.html#torch.Tensor.view'><strong><tt>view()</tt></strong></a> oraz <a href='https://pytorch.org/docs/master/torch.html#torch.reshape'><strong><tt>reshape()</tt></strong></a> w zasadzie robią to samo, zwracając tensor o zmienionym kształcie bez modyfikowania oryginału.<br>
Dobrą dyskusję na temat różnic znajdziesz <a href='https://stackoverflow.com/questions/49643225/whats-the-difference-between-reshape-and-view-in-pytorch'>tutaj</a>.


In [27]:
x = torch.arange(10)
print(x)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [28]:
x.view(2,5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [29]:
x.view(5,2)

tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])

In [30]:
# x pozostaje bez zmian
x

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### Widoki odzwierciedlają aktualne dane


In [31]:
z = x.view(2,5)
x[0]=234
print(z)

tensor([[234,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9]])


### Widoki mogą samodzielnie wywnioskować rozmiar
Przekazując <tt>-1</tt> sprawiamy, że PyTorch wyliczy właściwą wartość na podstawie danego tensora.


In [32]:
x.view(2,-1)

tensor([[234,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9]])

In [33]:
x.view(-1,5)

tensor([[234,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9]])

### Przyjmij kształt innego tensora dzięki <tt>.view_as()</tt>
<a href='https://pytorch.org/docs/master/tensors.html#torch.Tensor.view_as'><strong><tt>view_as(input)</tt></strong></a> działa tylko dla tensorów o tej samej liczbie elementów.


In [34]:
x.view_as(z)

tensor([[234,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9]])

## Arytmetyka tensorów
Dodawanie tensorów można wykonać na kilka sposobów, w zależności od oczekiwanego wyniku.<br>

Jako proste wyrażenie:


In [35]:
a = torch.tensor([1,2,3], dtype=torch.float)
b = torch.tensor([4,5,6], dtype=torch.float)
print(a + b)

tensor([5., 7., 9.])


Jako argumenty przekazane do operacji PyTorch:


In [36]:
print(torch.add(a, b))

tensor([5., 7., 9.])


Przekazując tensor wyjściowy jako argument:


In [37]:
result = torch.empty(3)
torch.add(a, b, out=result)  # to samo co result=torch.add(a,b)
print(result)

tensor([5., 7., 9.])


Modyfikacja tensora w miejscu


In [38]:
a.add_(b)  # to samo co a=torch.add(a,b)
print(a)

tensor([5., 7., 9.])


<div class="alert alert-info"><strong>UWAGA:</strong> Każda operacja, która zmienia tensor w miejscu, ma na końcu znak podkreślenia _.<br>
W powyższym przykładzie: <tt>a.add_(b)</tt> zmieniło tensor <tt>a</tt>.</div>


<table style="display: inline-block">
<caption style="text-align: center"><strong>Operacje arytmetyczne</strong></caption>
<tr><th>OPERACJA</th><th>FUNKCJA</th><th>OPIS</th></tr>
<tr><td>a + b</td><td>a.add(b)</td><td>dodawanie element po elemencie</td></tr>
<tr><td>a - b</td><td>a.sub(b)</td><td>odejmowanie</td></tr>
<tr><td>a * b</td><td>a.mul(b)</td><td>mnożenie</td></tr>
<tr><td>a / b</td><td>a.div(b)</td><td>dzielenie</td></tr>
<tr><td>a % b</td><td>a.fmod(b)</td><td>reszta z dzielenia</td></tr>
<tr><td>a<sup>b</sup></td><td>a.pow(b)</td><td>potęgowanie</td></tr>
<tr><td>&nbsp;</td><td></td><td></td></tr>
</table>


<table style="display: inline-block">
<caption style="text-align: center"><strong>Operacje monominalne</strong></caption>
<tr><th>OPERACJA</th><th>FUNKCJA</th><th>OPIS</th></tr>
<tr><td>|a|</td><td>torch.abs(a)</td><td>wartość bezwzględna</td></tr>
<tr><td>1/a</td><td>torch.reciprocal(a)</td><td>odwrotność</td></tr>
<tr><td>$\sqrt{a}$</td><td>torch.sqrt(a)</td><td>pierwiastek kwadratowy</td></tr>
<tr><td>log(a)</td><td>torch.log(a)</td><td>logarytm naturalny</td></tr>
<tr><td>e<sup>a</sup></td><td>torch.exp(a)</td><td>funkcja wykładnicza</td></tr>
<tr><td>12.34  ==>  12.</td><td>torch.trunc(a)</td><td>część całkowita</td></tr>
<tr><td>12.34  ==>  0.34</td><td>torch.frac(a)</td><td>część ułamkowa</td></tr>
</table>


<table style="display: inline-block">
<caption style="text-align: center"><strong>Trygonometria</strong></caption>
<tr><th>OPERACJA</th><th>FUNKCJA</th><th>OPIS</th></tr>
<tr><td>sin(a)</td><td>torch.sin(a)</td><td>sinus</td></tr>
<tr><td>cos(a)</td><td>torch.cos(a)</td><td>cosinus</td></tr>
<tr><td>tan(a)</td><td>torch.tan(a)</td><td>tangens</td></tr>
<tr><td>arcsin(a)</td><td>torch.asin(a)</td><td>arcus sinus</td></tr>
<tr><td>arccos(a)</td><td>torch.acos(a)</td><td>arcus cosinus</td></tr>
<tr><td>arctan(a)</td><td>torch.atan(a)</td><td>arcus tangens</td></tr>
<tr><td>sinh(a)</td><td>torch.sinh(a)</td><td>sinus hiperboliczny</td></tr>
<tr><td>cosh(a)</td><td>torch.cosh(a)</td><td>cosinus hiperboliczny</td></tr>
<tr><td>tanh(a)</td><td>torch.tanh(a)</td><td>tangens hiperboliczny</td></tr>
</table>


<table style="display: inline-block">
<caption style="text-align: center"><strong>Statystyki opisowe</strong></caption>
<tr><th>OPERACJA</th><th>FUNKCJA</th><th>OPIS</th></tr>
<tr><td>$\sum a$</td><td>torch.sum(a)</td><td>suma</td></tr>
<tr><td>$ar a$</td><td>torch.mean(a)</td><td>średnia</td></tr>
<tr><td>a<sub>max</sub></td><td>torch.max(a)</td><td>maksimum</td></tr>
<tr><td>a<sub>min</sub></td><td>torch.min(a)</td><td>minimum</td></tr>
<tr><td colspan="3">torch.max(a,b) zwraca tensor rozmiaru a<br>zawierający wartości maksymalne element po elemencie między a i b</td></tr>
</table>


<div class="alert alert-info"><strong>UWAGA:</strong> Większość operacji arytmetycznych wymaga wartości zmiennoprzecinkowych. Te, które działają na liczbach całkowitych, zwracają tensory całkowitoliczbowe.<br>
Na przykład <tt>torch.div(a,b)</tt> dla typów całkowitych wykonuje dzielenie całkowite (ucięcie części ułamkowej), a dla floatów – klasyczne dzielenie.</div>


#### Użyj poniższej przestrzeni, aby poeksperymentować z różnymi operacjami


In [51]:
a = torch.tensor([1,2,3], dtype=torch.float)
b = torch.tensor([4,5,6], dtype=torch.float)
print(torch.add(a,b).sum())

tensor(21.)


## Iloczyny skalarne
<a href='https://en.wikipedia.org/wiki/Dot_product'>Iloczyn skalarny</a> to suma iloczynów odpowiadających sobie elementów dwóch tensorów 1D. Jeśli tensory są wektorami, iloczyn skalarny ma postać:<br>

$\begin{bmatrix} a & b & c \end{bmatrix} \;\cdot\; \begin{bmatrix} d & e & f \end{bmatrix} = ad + be + cf$

Jeśli tensor zawiera wektor kolumnowy, wynik to suma elementów z pomnożonych macierzy. Przykładowo:<br>
$\begin{bmatrix} a & b & c \end{bmatrix} \;\cdot\; \begin{bmatrix} d \ e \ f \end{bmatrix} = ad + be + cf$<br><br>

Iloczyn skalarny można obliczyć funkcją <a href='https://pytorch.org/docs/stable/torch.html#torch.dot'><strong><tt>torch.dot(a,b)</tt></strong></a> lub zapisami `a.dot(b)` czy `b.dot(a)`


In [40]:
a = torch.tensor([1,2,3], dtype=torch.float)
b = torch.tensor([4,5,6], dtype=torch.float)
print(a.mul(b)) # dla porównania
print()
print(a.dot(b))

tensor([ 4., 10., 18.])

tensor(32.)


<div class="alert alert-info"><strong>UWAGA:</strong> Istnieje niewielka różnica między <tt>torch.dot()</tt> a <tt>numpy.dot()</tt>. <tt>torch.dot()</tt> przyjmuje wyłącznie argumenty 1D i zwraca iloczyn skalarny, natomiast <tt>numpy.dot()</tt> obsługuje także argumenty 2D i wykonuje mnożenie macierzy. Mnożenie macierzy pokazujemy poniżej.</div>


## Mnożenie macierzy
Dwuwymiarowe <a href='https://en.wikipedia.org/wiki/Matrix_multiplication'>mnożenie macierzy</a> jest możliwe, gdy liczba kolumn tensora <strong><tt>A</tt></strong> odpowiada liczbie wierszy tensora <strong><tt>B</tt></strong>. W takim przypadku iloczyn tensora <strong><tt>A</tt></strong> o rozmiarze $(x,y)$ i tensora <strong><tt>B</tt></strong> o rozmiarze $(y,z)$ daje tensor o rozmiarze $(x,z)$.
<div>
<div align="left"><img src='../_img/Matrix_multiplication_diagram.png' align="left"><br><br>

$\begin{bmatrix} a & b & c \
d & e & f \end{bmatrix} \;	imes\; \begin{bmatrix} m & n \ p & q \ r & s \end{bmatrix} = \begin{bmatrix} (am+bp+cr) & (an+bq+cs) \
(dm+ep+fr) & (dn+eq+fs) \end{bmatrix}$</div></div>

<div style="clear:both">Źródło obrazu: <a href='https://commons.wikimedia.org/wiki/File:Matrix_multiplication_diagram_2.svg'>https://commons.wikimedia.org/wiki/File:Matrix_multiplication_diagram_2.svg</a></div>

Mnożenie macierzy można obliczyć funkcją <a href='https://pytorch.org/docs/stable/torch.html#torch.mm'><strong><tt>torch.mm(a,b)</tt></strong></a> lub zapisami `a.mm(b)` albo `a @ b`


In [42]:
a = torch.tensor([[0,2,4],[1,3,5]], dtype=torch.float)
b = torch.tensor([[6,7],[8,9],[10,11]], dtype=torch.float)

print('a: ',a.size())
print('b: ',b.size())
print('a x b: ',torch.mm(a,b).size())

a:  torch.Size([2, 3])
b:  torch.Size([3, 2])
a x b:  torch.Size([2, 2])


In [43]:
print(torch.mm(a,b))

tensor([[56., 62.],
        [80., 89.]])


In [44]:
print(a.mm(b))

tensor([[56., 62.],
        [80., 89.]])


In [45]:
print(a @ b)

tensor([[56., 62.],
        [80., 89.]])


### Mnożenie macierzy z broadcastingiem
Mnożenie macierzy z <a href='https://pytorch.org/docs/stable/notes/broadcasting.html#broadcasting-semantics'>broadcastingiem</a> można obliczyć przy użyciu <a href='https://pytorch.org/docs/stable/torch.html#torch.matmul'><strong><tt>torch.matmul(a,b)</tt></strong></a> lub zapisów `a.matmul(b)` czy `a @ b`


In [46]:
t1 = torch.randn(2, 3, 4)
t2 = torch.randn(4, 5)

print(torch.matmul(t1, t2).size())

torch.Size([2, 3, 5])


Ten sam zabieg wywołuje jednak <tt><strong>RuntimeError</strong></tt> przy użyciu <tt>torch.mm()</tt>:


In [47]:
print(torch.mm(t1, t2).size())

RuntimeError: self must be a matrix

___
# Operacje zaawansowane


## Norma L2 (euklidesowa)
Zobacz <a href='https://pytorch.org/docs/stable/torch.html#torch.norm'><strong><tt>torch.norm()</tt></strong></a>

<a href='https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm'>Norma euklidesowa</a> daje normę wektora $x$, dla $x=(x_1,x_2,...,x_n)$.
Obliczamy ją jako

${\displaystyle \left\|\boldsymbol{x}\right\|_{2} := \sqrt{x_{1}^{2} + \cdots + x_{n}^{2}}}$


Po zastosowaniu do macierzy <tt>torch.norm()</tt> zwraca domyślnie <a href='https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm'>normę Frobeniusa</a>.


In [49]:
x = torch.tensor([2.,5.,8.,14.])
x.norm()

tensor(17.)

## Liczba elementów

Zobacz <a href='https://pytorch.org/docs/stable/torch.html#torch.numel'><strong><tt>torch.numel()</tt></strong></a>

Zwraca liczbę elementów w tensorze.


In [50]:
x = torch.ones(3,7)
x.numel()

21

Może być to przydatne w obliczeniach, takich jak błąd średniokwadratowy (MSE):<br>
<tt>
def mse(t1, t2):<br>
&nbsp;&nbsp;&nbsp;&nbsp;diff = t1 - t2<br>
    &nbsp;&nbsp;&nbsp;&nbsp;return torch.sum(diff * diff) / diff<strong>.numel()</strong></tt>


## Świetna robota!
